In [3]:
import igl
import scipy as sp
import numpy as np
import meshplot as mp

import os

os.chdir("/home/jovyan")
root_folder = os.getcwd()

## Assignment: spectral mesh filtering

The goal here is to filter a mesh using the spectral approach discussed in the slides. That is, the mesh is projected onto a limited number of eigenvectors of the Laplacian, which correspond to the low frequencies. Higher frequencies (eigenvectors corresponding to larger eigenvalues) are ignored. 

Change the code such that in each iteration step, `v_smooth` is a mesh that is filtered using the `n_ev[i]` number of eigenvectors. Note: this can be done in just one line of code.

In [5]:
# Spectral filtering using uniform Laplacian
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "cow.off"))

# uniform Laplacian using the mesh adjacency matrix; this is symmetric, so eigenvectors are orthogonal
adj = igl.adjacency_matrix(f)
l_unif = adj - sp.sparse.diags(adj.sum(0).A[0,:])

# try different numbers of eigenvectors for filtering
n_ev = [256,128,64,32,16,8]
v_smooth = [v]

for i in range(len(n_ev)):
    # compute the eigenvectors
    ew, ev = sp.sparse.linalg.eigsh(l_unif,n_ev[i],which='SM')
    # project the vertex positions onto the basis given by the eigenvectors
    # NEED TO REPLACE THE FOLLOWING LINE, WHICH IS JUST A PLACEHOLDER. SEE SLIDE 24
    v_smooth.append(v)
    
p = mp.plot(v_smooth[0], f, return_plot=True)

@mp.interact(level=(0, len(n_ev)))
def mcf(level=0):
    p.update_object(vertices=v_smooth[level])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='level', max=6), Output()), _dom_classes=('widget-interac…

## Assignment: Minimal surface by minimizing Dirichlet energy via the Laplace-Beltrami operator

The goal here is to compute a minimal surface under a fixed boundary by minimizing the Dirichlet energy. This can be done by making sure that the Laplace-Beltrami of the surface is zero everywhere in the interior (see slide 9). The algorithm proceeds iteratively, by updating the mesh to set the Laplace-Beltrami to zero, and then recomputing Laplace-Beltrami using the new mesh. For a much more detailed discussion of this approach, see [this paper by Pinkall et al](http://www.cs.jhu.edu/~misha/Fall09/Pinkall93.pdf). 

Change the code below such that in each step in the loop, the interior vertices `z_in` are computed by solving the [Laplace equation](https://en.wikipedia.org/wiki/Laplace%27s_equation). Follow the [example in the libigl tutorial](https://libigl.github.io/libigl-python-bindings/tutorials/#laplace-equation) that describes how to solve the Laplace equation with boundary conditions.

In [7]:
## Prepare the geometry first

# number of segments
n = 51

# a grid of s,t values
s = np.tile(np.linspace(-1, 1, n, endpoint=True),(n,1)).reshape(n*n,1)
t = np.tile(np.linspace(-1, 1, n, endpoint=True),(n,1)).transpose().reshape(n*n,1)

# 2D indices for vertices, reshaped into 1D arrays
ind_s = np.tile(np.arange(n-1),(n-1,1)).reshape((n-1)*(n-1),1)
ind_t = np.tile(np.arange(n-1),(n-1,1)).transpose().reshape((n-1)*(n-1),1)

# triangle connectivity
f_0 = np.concatenate((ind_s*n+ind_t, ind_s*n+ind_t+1, (ind_s+1)*n+ind_t),1)
f_1 = np.concatenate((ind_s*n+ind_t+1, (ind_s+1)*n+ind_t+1, (ind_s+1)*n+ind_t),1)
f = np.concatenate((f_0, f_1),0)

# vertex positions on a plane
x = s
y = t
z = np.zeros(s.shape)
v = np.concatenate((x,y,z),1)

# get indices of boundary vertices
v_b = np.unique(igl.boundary_facets(f))

# set some (non-planar) boundary conditions
v[v_b[s[v_b,0]>0.0],2] = np.abs(s[v_b[s[v_b,0]>0.0],0])
v[v_b[s[v_b,0]<0.0],2] = np.abs(s[v_b[s[v_b,0]<0.0],0])

## Minimal surface computation

# list of all vertex indices
v_all = np.arange(v.shape[0])

# list of interior indices
v_in = np.setdiff1d(v_all, v_b)

# iteration steps
v_min = [v.copy()]
steps = 256
for i in range(steps):
    # Laplace-Beltrami of current mesh
    l = igl.cotmatrix(v, f)
    
    # UPDATE THE MESH VERTICES HERE BY SOLVING THE LAPLACE EQUATION WITH BOUNDARY CONDITIONS
    # SEE https://libigl.github.io/libigl-python-bindings/tutorials/#laplace-equation
    # YOUR CODE HERE...
    
    # v are now the updated vertex positions; append them for visualization
    v_min.append(v.copy())
        
p = mp.plot(v_min[0], f, return_plot=True)

@mp.interact(level=(0, steps))
def mcf(level=0):
    p.update_object(vertices=v_min[level])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='level', max=256), Output()), _dom_classes=('widget-inter…